# Logistic Regression

In [1]:
import math
import numpy as np
import pandas as pd
import sklearn.linear_model
import statsmodels.api as sm
import matplotlib.pyplot as plt

## Simulierte Daten

In [2]:
def generate_prices(observations_count, prices_range):
    return np.random.choice(prices_range, observations_count)

def generate_competitor_prices(observations_count, competitors_count, prices_range):
    return np.matrix([generate_prices(observations_count, prices_range) for i in range(0, competitors_count)])
    
def calculate_ranks(observations_count, competitors_count, competitor_prices, prices):
    return [1 + len([1 for j in range(competitors_count) if competitor_prices[j,i] < prices[i]])
            for i in range(observations_count)]

def calculate_sold_probs_A(observations_count, ranks, competitors_count, prices):
    max_prob = lambda i: 1 - ((0.3 * ranks[0][i])/(competitors_count[0] + 1)) - 0.05 * prices[0][i] + (-0.0125 * (prices[0][i] - prices[1][i]) + 0.25)
    return [np.maximum(0, np.round(np.random.uniform(0, max_prob(i)))) for i in range(observations_count)]

def calculate_sold_probs_B(observations_count, ranks, competitors_count, prices):
    max_prob = lambda i: 1 - ((0.3 * ranks[1][i])/(competitors_count[1] + 1)) - 0.05 * prices[1][i] + (0.0125 * (prices[0][i] - prices[1][i]) + 0.25)
    return [np.maximum(0, np.round(np.random.uniform(0, max_prob(i)))) for i in range(observations_count)]

observations_count = 1000
prices_ranges = [np.arange(1, 20.1, 0.1) for i in range(2)] #TODO adopt prob function if price changes
prices = [generate_prices(observations_count, prices_ranges[i]) for i in range(2)] # TODO p1 > p2?
competitors_count = [5, 5]
competitor_prices = [generate_competitor_prices(observations_count, competitors_count[i], prices_ranges[i]) for i in range(2)]
ranks = [calculate_ranks(observations_count, competitors_count[i], competitor_prices[i], prices[i]) for i in range(2)]
sold = [calculate_sold_probs_A(observations_count, ranks, competitors_count, prices),
        calculate_sold_probs_B(observations_count, ranks, competitors_count, prices)]

## Regression

In [3]:
def get_explanatory_vars_A(observations_count, competitors_count, ranks, competitor_prices):
    explanatory_1 = [1 for j in range(observations_count)]
    explanatory_2 = [ranks[0][j] for j in range(observations_count)]
    
    def get_all_competitor_prices(j):
        for i in range(2):
            for k in range(competitors_count[i]):
                yield competitor_prices[i][k,j]
    explanatory_3 = [prices[0][j] - np.min(list(get_all_competitor_prices(j))) for j in range(observations_count)]
    explanatory_4 = [prices[0][j] - np.min([prices[i][j] for i in range(2)]) for j in range(observations_count)]
    explanatory_5 = [math.pow(explanatory_4[j], 2) for j in range(observations_count)]
    return np.matrix([explanatory_1, explanatory_2, explanatory_3, explanatory_4, explanatory_5])

def get_explanatory_vars_B(observations_count, competitors_count, ranks, competitor_prices):
    explanatory_1 = [1 for j in range(observations_count)]
    explanatory_2 = [ranks[1][j] for j in range(observations_count)]
    
    def get_all_competitor_prices(j):
        for i in range(2):
            for k in range(competitors_count[i]):
                yield competitor_prices[i][k,j]
    explanatory_3 = [prices[1][j] - np.min(list(get_all_competitor_prices(j))) for j in range(observations_count)]
    explanatory_4 = [prices[1][j] - np.min([prices[i][j] for i in range(2)]) for j in range(observations_count)]
    explanatory_5 = [math.pow(explanatory_4[j], 2) for j in range(observations_count)]
    return np.matrix([explanatory_1, explanatory_2, explanatory_3, explanatory_4, explanatory_5])

explanatory_vars = [get_explanatory_vars_A(observations_count, competitors_count, ranks, competitor_prices),
                    get_explanatory_vars_B(observations_count, competitors_count, ranks, competitor_prices)]
logits = [sm.Logit(sold[i], explanatory_vars[i].transpose()) for i in range(2)]
results = [logits[i].fit() for i in range(2)]
beta = [results[i].params for i in range(2)]
beta

Optimization terminated successfully.
         Current function value: 0.341628
         Iterations 11
Optimization terminated successfully.
         Current function value: 0.378146
         Iterations 10


[array([ 0.87957805, -0.33554044, -0.19526857, -0.04787309, -0.02603562]),
 array([ 0.7827886 , -0.31014589, -0.1525739 , -0.17402205, -0.01771691])]

In [4]:
[results[i].aic for i in range(2)]

[693.2563437935587, 766.2924583262751]

## Optimierung

In [5]:
def get_price_index(price):
    return int(price / 0.1 - 10) # TODO dependent on price range?

def calculate_sale_probs(beta, explanatory_vars, prices_range):
    L = lambda price: np.sum([beta[m] * explanatory_vars[m,get_price_index(price)] for m in range(explanatory_vars.shape[0])])
    return [np.exp(L(price)) / (1 + np.exp(L(price))) for price in prices_range]

def bellman(n, sale_probs, prices_range, holding_cost_rate, delta, values, step):
    prob_A = lambda price, i: sale_probs[0, i, get_price_index(price)]
    prob_B = lambda price, j: sale_probs[1, j, get_price_index(price)]
    todays_profit = lambda prices, j, i: np.min([n[j], i]) * prices[j] - n[j] * holding_cost_rate[j]
    disc_exp_fut_profits = lambda i, j: delta * values[step + 1, np.max([0, n[0]-i]), np.max([0, n[1]-j])]

    bellman_for_combinations = []
    for price_A in prices_range[0]:
        for price_B in prices_range[1]:
            bellman_for_combinations.append(np.sum([prob_A(price_A, i) * np.sum([prob_B(price_B, j) * ((todays_profit([price_A, price_B], 0, i)+todays_profit([price_A, price_B], 1, j)) + disc_exp_fut_profits(i,j)) for j in range(2)]) for i in range(2)]))
    return np.max(bellman_for_combinations)

def bellman_check(n, sale_probs, prices, holding_cost_rate, delta, values, t):
    prob_A = lambda price, i: sale_probs[0, i, get_price_index(price)]
    prob_B = lambda price, j: sale_probs[1, j, get_price_index(price)]
    todays_profit = lambda prices, j, i: np.min([n[j], i]) * prices[j] - n[j] * holding_cost_rate[j]
    disc_exp_fut_profits = lambda i, j: delta * values[t + 1, np.max([0, n[0]-i]), np.max([0, n[1]-j])]

    return np.sum([prob_A(prices[0], i) * np.sum([prob_B(prices[1], j) * ((todays_profit(prices, 0, i)+todays_profit(prices, 1, j)) + disc_exp_fut_profits(i,j)) for j in range(2)]) for i in range(2)])

In [6]:
competitor_prices = [generate_prices(competitors_count[i],prices_ranges[i]) for i in range(2)]

def get_explanatory_vars(prices_range, competitors_count, competitor_prices):
    data = []
    for i in range(3):
        if i == 0:
            data.append([1 for price in prices_range])
        elif i == 1:
            data.append([1 + len([1 for j in range(0, competitors_count) if competitor_prices[j] < price]) for price in prices_range])
        else:
            data.append([price - np.min([competitor_prices[j] for j in range(competitors_count)]) for price in prices_range])
    explanatory = np.matrix(data)
    return explanatory

explanatory_vars = [get_explanatory_vars(prices_ranges[i], competitors_count[i], competitor_prices[i]) for i in range(2)]

In [7]:
p = [calculate_sale_probs(beta[i], explanatory_vars[i], prices_ranges[i]) for i in range(2)]
sale_probs = np.array([[[1 - j for j in p[i]], p[i]] for i in range(2)])

delta = 0.99
holding_cost_rate = [0.01, 0.01]
init_inventory = [3, 3]
steps = 50
values = np.empty(shape=(steps + 1, init_inventory[0] + 1, init_inventory[1] + 1))
for step in range(steps, -1, -1):
    for n_A in range(init_inventory[0] + 1):
        for n_B in range(init_inventory[1] + 1):
            if step == steps or n_A == 0 or n_B == 0:
                values[step, n_A, n_B] = 0
            else:
                values[step, n_A, n_B] = bellman([n_A, n_B], sale_probs, prices_ranges, holding_cost_rate, delta, values, step)

def opt_p(t):
    opt_prices = np.zeros(shape=(init_inventory[0] + 1, init_inventory[1] + 1, 2))
    for n_A in range(1, init_inventory[0] + 1):
        for n_B in range(1, init_inventory[1] + 1):
            for price_A in prices_ranges[0]:
                for price_B in prices_ranges[1]:
                    if bellman_check([n_A, n_B], sale_probs, [price_A, price_B], holding_cost_rate, delta, values, t) == values[t, n_A, n_B]:
                        opt_prices[n_A, n_B, 0] = price_A
                        opt_prices[n_A, n_B, 1] = price_B
    return opt_prices
                    
opt_prices = opt_p(0)
for n_A in range(init_inventory[0] + 1):
    for n_B in range(init_inventory[1] + 1):
        print('n_A: ' + str(n_A) + ',n_B: ' + str(n_B) + ' - A: ' + str(opt_prices[n_A, n_B, 0]) + ',B: ' + str(opt_prices[n_A, n_B, 1]))

n_A: 0,n_B: 0 - A: 0.0,B: 0.0
n_A: 0,n_B: 1 - A: 0.0,B: 0.0
n_A: 0,n_B: 2 - A: 0.0,B: 0.0
n_A: 0,n_B: 3 - A: 0.0,B: 0.0
n_A: 1,n_B: 0 - A: 0.0,B: 0.0
n_A: 1,n_B: 1 - A: 17.6,B: 18.9
n_A: 1,n_B: 2 - A: 20.0,B: 15.8
n_A: 1,n_B: 3 - A: 20.0,B: 10.9
n_A: 2,n_B: 0 - A: 0.0,B: 0.0
n_A: 2,n_B: 1 - A: 12.4,B: 20.0
n_A: 2,n_B: 2 - A: 16.5,B: 17.6
n_A: 2,n_B: 3 - A: 17.4,B: 15.0
n_A: 3,n_B: 0 - A: 0.0,B: 0.0
n_A: 3,n_B: 1 - A: 10.1,B: 20.0
n_A: 3,n_B: 2 - A: 12.4,B: 18.9
n_A: 3,n_B: 3 - A: 12.4,B: 15.4


## Simulation

In [8]:
c_prices_time_1 = np.empty(shape=(competitors_count[0], steps+1))
for c in range(competitors_count[0]):
    c_prices_time_1[c,0] = competitor_prices[0][c]
c_prices_time_2 = np.empty(shape=(competitors_count[1], steps+1))
for c in range(competitors_count[1]):
    c_prices_time_2[c,0] = competitor_prices[1][c]
    
inventory_time_1 = np.empty(shape=(steps))
for t in range(steps):
    inventory_time_1[t] = init_inventory[0]
inventory_time_2 = np.empty(shape=(steps))
for t in range(steps):
    inventory_time_2[t] = init_inventory[1]

o_prices_time_1 = np.zeros(shape=(steps))
o_prices_time_2 = np.zeros(shape=(steps))
o_prices_time_1[0] = opt_prices[init_inventory[0], init_inventory[1]][0]
o_prices_time_2[0] = opt_prices[init_inventory[0], init_inventory[1]][1]

for t in range(1,steps):
    if ((inventory_time_1[t] > 0) and (np.random.uniform(0,1) < sale_probs[0, 1,get_price_index(o_prices_time_1[t-1])])):
        inventory_time_1[t] = np.max([0, inventory_time_1[t-1]-1])
    else:
        inventory_time_1[t] = inventory_time_1[t-1]
        
    if ((inventory_time_2[t] > 0) and (np.random.uniform(0,1) < sale_probs[1, 1,get_price_index(o_prices_time_2[t-1])])):
        inventory_time_2[t] = np.max([0, inventory_time_2[t-1]-1])
    else:
        inventory_time_2[t] = inventory_time_2[t-1]
        
    #for c in range(competitors_count):
        #if (np.random.uniform(0,1) < 0.2):
            #c_prices_time[c,t]= np.round(c_prices_time[c,t-1] * np.random.uniform(0.8, 1.2), 1)
            
    if (inventory_time_1[t] > 0) or (inventory_time_2[t] > 0):
        opt_prices_n = opt_p(t)
        o_prices_time_1[t] = opt_prices_n[int(inventory_time_1[t]), int(inventory_time_2[t])][0]
        o_prices_time_2[t] = opt_prices_n[int(inventory_time_1[t]), int(inventory_time_2[t])][1]
inventory_time_1

array([ 3.,  3.,  2.,  2.,  2.,  2.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [9]:
inventory_time_2

array([ 3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [10]:
o_prices_time_1

array([ 12.4,  12.4,  17.4,  17.4,  17.3,  17.3,  20. ,  20. ,  20. ,
        20. ,  20. ,  20. ,  20. ,  20. ,  20. ,  20. ,  20. ,  20. ,
        20. ,  20. ,  20. ,  20. ,  20. ,  20. ,  20. ,  20. ,  20. ,
        20. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
         0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
         0. ,   0. ,   0. ,   0. ,   0. ])

In [11]:
o_prices_time_2

array([ 15.4,  15.3,  14.9,  14.9,  14.9,  14.8,  10.9,  10.9,  10.9,
        10.9,  10.9,  10.9,  10.9,  10.9,  10.9,  10.9,  10.9,  10.9,
        15.2,  15.1,  15.1,  15. ,  15. ,  14.9,  14.9,  14.8,  10.9,
        10.9,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
         0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
         0. ,   0. ,   0. ,   0. ,   0. ])